In [2]:
# ALL IMPORTS
import pandas as pd
import networkx as nx
import numpy as np
import io
import re
import pickle

In [3]:
def read_file(countries, athletes, events):
    """ This functions takes the 3 .csv files --> 
    Convert them into a panda dataframe -->
    Do some datacleaning --> 
    add the WikiLink column
    
    Input: 3x .csv files
    -------------------------------
    Output: 3x dataframes
    """
    df_countries = pd.read_csv(countries, sep = ';',   encoding = 'latin-1', dtype = object) # Convert countries.csv into panda dataframe
    df_countries['WikiLink'] = df_countries['country']+"_at_the_2016_Summer_Olympics"        # Add a column for WikiLinks
    df_countries['country2'] = df_countries['country']                                       # Add a column country2 (Will be used for finding athletes WikiLink)
    df_countries['country2'] = df_countries['country2'].str.replace('\s', '_')               # Replase space with _
    df_countries['WikiLink'] = df_countries['WikiLink'].str.replace('\s', '_')               # Replase space with _
    df_countries['WikiLink'] = df_countries['WikiLink'].str.replace("*", "")                 # Remove *
    df_countries = df_countries.dropna()                                                     # Remove NanN
    
    df_athletes = pd.read_csv(athletes, sep = ',',   encoding = 'utf-8', dtype = object)     # Convert athletes.csv to panda dataframe
    df_athletes['name2'] = df_athletes['name']                                               # Add a column name2 (Will be used for finding athletes WikiLink)
    df_athletes['name2'] = df_athletes['name2'].str.replace('\s', '_')                       # Replase space with _
    df_athletes = df_athletes.dropna()                                                       # Remove NanN
    
    df_events = pd.read_csv(events, sep = ';', encoding = 'latin-1', dtype = object)         # Convert events.csv to panda dataframe
    df_events['WikiLink'] = df_events['sport']+"_at_the_2016_Summer_Olympics"                # Add a column for WikiLinks
    df_events['WikiLink'] = df_events['WikiLink'].str.replace('canoe', 'canoeing')           # Replace canou (for Matching Wikilink)
    df_events['WikiLink'] = df_events['WikiLink'].str.replace('hockey', 'field hockey')      # Replace hockey (for Matching Wikilink)
    df_events['WikiLink'] = df_events['WikiLink'].str.replace('synchronised','synchronized') # Replace synchronised (for Matching Wikilink)
    df_events['WikiLink'] = df_events['WikiLink'].str.replace('\s', '_')                     # Replase space with _
    df_events = df_events.drop_duplicates(subset = ["WikiLink"])                             # drop duplicates
    df_events = df_events.dropna()                                                           # Remove NanN
    
    return df_countries, df_athletes, df_events

df_countries,df_athletes, df_events = read_file('countries.csv','athletes.csv','events.csv')


def Athlet_WikiLink(df1, df2):
    """ This functions finds the athletes wikilinks by 
    find all wikilinks from the coutries wikipages. 
    If the wikilinks matches a name from the athletes dataframe
    save the wikilink in a list.
    
    Input: athletes dataframe, countries dataframe
    -------------------------------
    Output: list with athletes Wikilinks
    """
    A_WikiLink = []                                                                          # Create an empty list
    path_folder = ("./Files/")                                                               # Folder with all the countries json wikipages
    for i in df2['country2']:                                                                # Run through all countries
        f = io.open(path_folder + 'countries_'+ i + '_at_the_2016_Summer_Olympics.txt','r',encoding = 'utf-8').read() # Open the file
        links = re.findall("\[\[(.*?)\]\]", f)                                               # Find all links
        links = [x.replace(' ','_') for x in links]                                          # Replase space with _
        links = [s.split('|') for s in links]                                                # Split the links by the '|'

        for j in range(len(links)):                                                          # Run through all links
                if len(links[j]) == 1:                                                       # If the name and the links are the same
                    if len(df1.loc[df1['name2'] == links[j][0]]) >= 1:                       # If the link excist in the athletes dataframe
                        A_WikiLink.append([df1.at[df1.loc[df1['name2'] == links[j][0]].index[0],'id'],links[j][0]]) # append the wikilink and the 'id' number to the list
                elif len(links[j]) == 2:                                                     # If the name and the links are not the same
                    if len(df1.loc[df1['name2'] == links[j][1]]) >= 1:                       # If the link excist in the athletes dataframe
                        A_WikiLink.append([df1.at[df1.loc[df1['name2'] == links[j][1]].index[0],'id'],links[j][0]]) # append the wikilink and the 'id' number to the list
    return A_WikiLink

A_WikiLink = Athlet_WikiLink(df_athletes,df_countries)           # Run Function

df_if = pd.DataFrame(A_WikiLink, columns =['id', 'WikiLink'])    # Convert wikilink list into panda dataframe
df_athletes = df_athletes.merge(df_if)                           # Merge wikilink dataframe and athletes dataframe
df_athletes = df_athletes.drop_duplicates()                      # Drop duplicates


In [4]:
# Save the dataframes:
pickle.dump(df_athletes, open('df_athletes.txt', 'wb'))
pickle.dump(df_countries, open('df_countries.txt', 'wb'))
pickle.dump(df_events, open('df_events.txt', 'wb'))